**DeapSECURE module 5: Cryptography for Privacy-Preserving Computation (part A: Data Protection)**

# Session 2 Solution: AES Encryption and Decryption

This notebook provides complete solutions for AES encryption and decryption exercises,
including practical examples with both educational and production-grade implementations.

## 1. Setup and Helper Functions

In [ ]:
import time
import numpy as np

In [ ]:
def encode_int(C, minlength=16):
    """Encodes an arbitrarily long integer into a bytes object."""
    C_hex = hex(C)[2:]
    if len(C_hex) % 2:
        C_hex = '0' + C_hex
    C_bytes = bytes.fromhex(C_hex)
    if len(C_bytes) < minlength:
        C_bytes = C_bytes.rjust(minlength, b'\x00')
    return C_bytes

def decode_int(B):
    """Decodes a bytes object into a long integer."""
    return int(B.hex(), 16)

def leftpad16(B):
    """Pad a bytes array from the left with NULL chars to multiple of 16 bytes."""
    padlength = len(B) % 16
    if padlength > 0:
        return (b'\x00' * (16 - padlength)) + B
    else:
        return B

## 2. Educational AES Implementation (aes.py)

The aes module provides a simple, pure-Python implementation for learning purposes.

In [ ]:
# Import the educational AES module
import aes

### 2.1 Key Generation and Setup

In [ ]:
# Define a secret key (must be less than 128 bits / 16 bytes)
secret_key = 0x5e413c
print(f"Secret key (hex): {hex(secret_key)}")
print(f"Secret key (decimal): {secret_key}")

# Create AES encryptor/decryptor object
E = aes.AES(secret_key)
print(f"\nAES object created successfully")

### 2.2 Plaintext Definition

In [ ]:
# Define plaintext (must be under 16 bytes)
plaintext_string = 'IdeaFusion'
plaintext = plaintext_string.encode()  # UTF-8 encoding
plaintext_int = decode_int(plaintext)

print(f"Plaintext string: '{plaintext_string}'")
print(f"Plaintext bytes: {plaintext}")
print(f"Plaintext (decimal): {plaintext_int}")
print(f"Plaintext (hex): {hex(plaintext_int)}")

### 2.3 Encryption

In [ ]:
# Encrypt the plaintext
ciphertext_int = E.encrypt(plaintext_int)

print(f"Ciphertext (decimal): {ciphertext_int}")
print(f"Ciphertext (hex): {hex(ciphertext_int)}")

# Convert to bytes
ciphertext_bytes = encode_int(ciphertext_int)
print(f"Ciphertext (bytes): {ciphertext_bytes}")
print(f"Ciphertext (hex bytes): {ciphertext_bytes.hex()}")

In [ ]:
# Analysis of ciphertext
print(f"Original message length: {len(plaintext)} bytes")
print(f"Encrypted message length: {len(ciphertext_bytes)} bytes")
print(f"\nNote: Both are 16 bytes because AES operates on 16-byte blocks")
print(f"The ciphertext appears as random bytes (not readable)")

### 2.4 Decryption with Correct Key

In [ ]:
# Decrypt with the correct key
decrypted_int = E.decrypt(ciphertext_int)
decrypted_bytes = encode_int(decrypted_int)

print(f"Decrypted (hex): {hex(decrypted_int)}")
print(f"Decrypted (bytes): {decrypted_bytes}")

# Remove padding (leading NULLs)
decrypted_message = decrypted_bytes.lstrip(b'\x00')
print(f"\nDecrypted message: {decrypted_message}")
print(f"Matches original: {decrypted_message == plaintext}")

### 2.5 Decryption with Wrong Key

In [ ]:
# Create AES object with wrong key
wrong_key = 0x27F6A123
E_wrong = aes.AES(wrong_key)

# Try to decrypt with wrong key
wrong_decrypted_int = E_wrong.decrypt(ciphertext_int)
wrong_decrypted_bytes = encode_int(wrong_decrypted_int)
wrong_message = wrong_decrypted_bytes.lstrip(b'\x00')

print(f"Decrypted with wrong key: {wrong_message}")
print(f"Matches original: {wrong_message == plaintext}")
print(f"\nConclusion: Without the correct key, decryption produces garbage!")

In [ ]:
# Try multiple wrong keys
print("Attempting decryption with various wrong keys:")
for i in range(5):
    wrong_key_i = secret_key + i + 1
    E_i = aes.AES(wrong_key_i)
    decrypted_i = E_i.decrypt(ciphertext_int)
    msg_i = encode_int(decrypted_i).lstrip(b'\x00')
    print(f"Key {hex(wrong_key_i)}: {msg_i}")

### 2.6 Encryption with Different Keys

In [ ]:
# Encrypt with similar keys
print(f"Original key: {hex(secret_key)}")
print(f"Original ciphertext: {hex(ciphertext_int)}\n")

for i in range(1, 4):
    key_i = secret_key + i
    E_i = aes.AES(key_i)
    cipher_i = E_i.encrypt(plaintext_int)
    print(f"Key {hex(key_i)}: {hex(cipher_i)}")

print("\nObservation: Even slightly different keys produce completely different ciphertexts!")
print("This is the avalanche effect - a key property of secure encryption.")

### 2.7 Performance Analysis

In [ ]:
# Measure encryption performance
print("Encryption Performance (aes.py - pure Python):")
%timeit -n 100 -r 5 E.encrypt(plaintext_int)

In [ ]:
# Measure decryption performance
print("Decryption Performance (aes.py - pure Python):")
%timeit -n 100 -r 5 E.decrypt(ciphertext_int)

In [ ]:
# Key cracking time estimation
print("Key Cracking Time Estimates:")
print("\nAssuming 0.2 seconds per 1000 encryptions (0.0002 sec/encryption):\n")

key_sizes = [8, 16, 24, 32]
for bits in key_sizes:
    combinations = 2 ** bits
    time_seconds = combinations * 0.0002
    time_hours = time_seconds / 3600
    time_days = time_hours / 24
    time_years = time_days / 365
    
    print(f"{bits}-bit key: {combinations:,} combinations")
    if time_seconds < 60:
        print(f"  Time: {time_seconds:.2f} seconds")
    elif time_hours < 24:
        print(f"  Time: {time_hours:.2f} hours")
    elif time_days < 365:
        print(f"  Time: {time_days:.2f} days")
    else:
        print(f"  Time: {time_years:.2f} years")
    print()

## 3. Production-Grade AES (PyCryptodome)

PyCryptodome provides optimized, production-ready cryptographic implementations.

In [ ]:
# Import PyCryptodome AES
from Crypto.Cipher import AES

### 3.1 ECB Mode (Educational - Not Recommended for Production)

In [ ]:
# Create secret key in bytes format
secret_bkey = encode_int(secret_key)
print(f"Secret key (hex): {secret_bkey.hex()}")
print(f"Secret key (bytes): {secret_bkey}")

# Create AES cipher in ECB mode
EE = AES.new(secret_bkey, AES.MODE_ECB)
print(f"\nAES cipher created in ECB mode")

In [ ]:
# Encrypt with PyCryptodome
plaintext_padded = leftpad16(b'IdeaFusion')
ciphertext_pycrypto = EE.encrypt(plaintext_padded)

print(f"Plaintext (padded): {plaintext_padded}")
print(f"Ciphertext: {ciphertext_pycrypto}")
print(f"Ciphertext (hex): {ciphertext_pycrypto.hex()}")

In [ ]:
# Decrypt with PyCryptodome
decrypted_pycrypto = EE.decrypt(ciphertext_pycrypto)
print(f"Decrypted: {decrypted_pycrypto}")
print(f"Decrypted (unpadded): {decrypted_pycrypto.lstrip(b'\x00')}")

In [ ]:
# Encrypt longer message
long_message = b'The master key (a secret) must be less than 128 bits (16 bytes)'
long_padded = leftpad16(long_message)
print(f"Original length: {len(long_message)} bytes")
print(f"Padded length: {len(long_padded)} bytes")

long_ciphertext = EE.encrypt(long_padded)
print(f"Ciphertext length: {len(long_ciphertext)} bytes")

long_decrypted = EE.decrypt(long_ciphertext)
print(f"\nDecrypted matches original: {long_decrypted.lstrip(b'\x00') == long_message}")

### 3.2 Performance Comparison

In [ ]:
# Performance: aes.py (pure Python)
print("Pure Python AES (aes.py):")
%timeit -n 100 -r 5 E.encrypt(plaintext_int)

In [ ]:
# Performance: PyCryptodome (C-optimized)
print("PyCryptodome AES (C-optimized):")
%timeit -n 1000 -r 5 EE.encrypt(plaintext_padded)

In [ ]:
print("\nConclusion: PyCryptodome is ~30-100x faster than pure Python!")
print("This is because performance-critical operations are implemented in C.")

### 3.3 EAX Mode (Authenticated Encryption - Recommended for Production)

In [ ]:
# Create AES cipher in EAX mode (authenticated encryption)
EAX = AES.new(secret_bkey, AES.MODE_EAX)
EAX_nonce = EAX.nonce

print(f"EAX nonce: {EAX_nonce.hex()}")
print(f"Nonce length: {len(EAX_nonce)} bytes")

In [ ]:
# Encrypt with EAX
message = leftpad16(b'The master key (a secret) must be less than 128 bits (16 bytes)')
ciphertext_eax = EAX.encrypt(message)

print(f"Ciphertext (EAX): {ciphertext_eax.hex()}")

In [ ]:
# Decrypt with EAX (need same nonce)
DD_EAX = AES.new(secret_bkey, AES.MODE_EAX, nonce=EAX_nonce)
decrypted_eax = DD_EAX.decrypt(ciphertext_eax)

print(f"Decrypted: {decrypted_eax.lstrip(b'\x00')}")

In [ ]:
# Demonstrate that EAX produces different ciphertexts each time (due to random nonce)
print("EAX Mode - Different Ciphertexts Each Time (due to random nonce):\n")

for i in range(3):
    EAX_i = AES.new(secret_bkey, AES.MODE_EAX)
    cipher_i = EAX_i.encrypt(message)
    print(f"Attempt {i+1}: {cipher_i.hex()[:32]}...")

print("\nNote: Each encryption produces different ciphertext (more secure!)")

## 4. Key Cracking Challenge

Demonstrates the computational difficulty of brute-force key recovery.

In [ ]:
# Create a challenge: encrypt a message with unknown key
challenge_key = 0xABCDEF  # Unknown to us
challenge_message = b'SECRET'

E_challenge = aes.AES(challenge_key)
challenge_plaintext_int = decode_int(challenge_message)
challenge_ciphertext = E_challenge.encrypt(challenge_plaintext_int)

print(f"Challenge ciphertext: {hex(challenge_ciphertext)}")
print(f"\nTask: Find the key that decrypts this message to a readable string")

In [ ]:
# Brute force attack on 16-bit key space
def is_valid_message(msg_bytes):
    """Check if decrypted message contains only printable ASCII characters."""
    msg = msg_bytes.lstrip(b'\x00')
    if len(msg) == 0:
        return False
    try:
        msg.decode('ascii')
        return all(32 <= b < 127 for b in msg)  # Printable ASCII
    except:
        return False

print("Attempting brute-force attack on 16-bit key space...\n")

start_time = time.time()
found = False

for key_attempt in range(0x10000):  # 16-bit key space
    E_attempt = aes.AES(key_attempt)
    decrypted = E_attempt.decrypt(challenge_ciphertext)
    decrypted_bytes = encode_int(decrypted)
    
    if is_valid_message(decrypted_bytes):
        msg = decrypted_bytes.lstrip(b'\x00')
        print(f"Found! Key: {hex(key_attempt)}")
        print(f"Message: {msg}")
        found = True
        break

end_time = time.time()
print(f"\nTime taken: {end_time - start_time:.2f} seconds")
if found:
    print(f"Correct key was: {hex(challenge_key)}")

## 5. Summary: Key Takeaways

Important lessons from AES encryption and decryption.

In [ ]:
print("="*70)
print("KEY TAKEAWAYS: AES ENCRYPTION AND DECRYPTION")
print("="*70)

print("""
1. ENCRYPTION BASICS:
   - AES operates on 16-byte blocks
   - Plaintext must be padded to multiple of 16 bytes
   - Same plaintext + same key = same ciphertext (in ECB mode)

2. KEY SECURITY:
   - Without the correct key, decryption produces garbage
   - Small key differences produce completely different ciphertexts
   - This is the "avalanche effect" - essential for security

3. PERFORMANCE:
   - Pure Python implementations are slow (~0.2ms per operation)
   - C-optimized libraries (PyCryptodome) are 30-100x faster
   - For production use, always use optimized libraries

4. MODES OF OPERATION:
   - ECB: Simple but not recommended (reveals patterns)
   - EAX: Authenticated encryption with random nonce (recommended)
   - Each mode has different security properties

5. KEY CRACKING:
   - 8-bit key: ~0.05 seconds to crack
   - 16-bit key: ~13 seconds to crack
   - 24-bit key: ~1 hour to crack
   - 128-bit key: Computationally infeasible (10^30+ years)

6. BEST PRACTICES:
   - Use well-established algorithms (AES, not custom ciphers)
   - Use authenticated encryption modes (EAX, GCM, CCM)
   - Use production-grade libraries (PyCryptodome, cryptography)
   - Never roll your own encryption unless you're an expert
   - Keep keys secure and use strong key derivation functions
""")

print("="*70)